In [57]:
import pandas as pd
import matplotlib.pyplot as plt


In [58]:
df=pd.read_csv('./data/sp500_stocks.csv')
df.shape

(1890274, 8)

It is pratical to convert the Date column of the dataset to an column of Timestamps, since it will make operations related to the dates much easier to iplement using pandas 

In [59]:
# Making sure the Date in the dataset is considered an date in pandas
print(f"Before: {type(df['Date'].loc[0])}")
df['Date']= pd.to_datetime(df['Date'])
print(f"After: {type(df['Date'].loc[0])}")


Before: <class 'str'>
After: <class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [60]:
df.head()

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
0,2010-01-04,MMM,43.783867,69.414719,69.774246,69.122070,69.473244,3640265.0
1,2010-01-05,MMM,43.509628,68.979935,69.590302,68.311035,69.230766,3405012.0
2,2010-01-06,MMM,44.126682,69.958191,70.735786,69.824417,70.133781,6301126.0
3,2010-01-07,MMM,44.158325,70.008362,70.033447,68.662209,69.665550,5346240.0
4,2010-01-08,MMM,44.469463,70.501671,70.501671,69.648827,69.974915,4073337.0


Nan values exploration:

In [61]:
columns_with_nan = df.columns[df.isna().any()].tolist()

print("Columns containing NaN values:", columns_with_nan)

Columns containing NaN values: ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']


It is possible to understand that the Nan values are probably associated to an error when getting the infomation since the Date column doesn't present any Nan values, that also gives more confidence in the continuity of our timeseries information

We are now going to see the magnitude of the ammount of Nan values 

In [62]:
n_rows_na= df.isna().any(axis=1).sum()
print(f"Number of rows with atleast one Nan value: {n_rows_na}")

Number of rows with atleast one Nan value: 101626


It is possible to see that we have an considerable ammount of missing values about 5% of our entire dataset is affected

In [63]:
df_without_na=df.dropna()

print(f"Original dataset shape: {df.shape}")
print(f"Dtaset without rows containing Nan shape: {df_without_na.shape}")

Original dataset shape: (1890274, 8)
Dtaset without rows containing Nan shape: (1788648, 8)


To have an better idea of how these missing values would affect our time series we will see how many gaps (days are missing from the ideal case in which there is an row for each day) exist

In [87]:
# Step 1: Create the full date range
full_date_range = pd.date_range(start=df['Date'].min(), end=df['Date'].max())

# Step 2: Count missing dates in the global dataset
def count_missing_dates(df):
    available_dates = df['Date'].unique()
    available_dates.shape
    full_date_range.shape
    missing_dates = set(full_date_range) - set(available_dates)
    return len(missing_dates)

missing_dates_original = count_missing_dates(df)
missing_dates_cleaned = count_missing_dates(df_without_na)

print(f"Total missing dates in the original dataset: {missing_dates_original}")
print(f"Total missing dates in the cleaned dataset: {missing_dates_cleaned}")

# Step 3: Group by 'Symbol' and count missing dates for each group
def count_missing_dates_by_symbol(grouped, full_date_range):
    results = {}
    for symbol, group in grouped:
        available_dates = group['Date'].dropna().unique()
        missing_dates = set(full_date_range) - set(available_dates)
        results[symbol] = len(missing_dates)
    return results

# Group the dataset by 'Symbol'
df_grouped = df.groupby('Symbol')
df_without_na_grouped = df_without_na.groupby('Symbol')

# Create per-symbol date ranges
missing_dates_by_symbol_original = count_missing_dates_by_symbol(df_grouped, full_date_range)
missing_dates_by_symbol_cleaned = count_missing_dates_by_symbol(df_without_na_grouped, full_date_range)

print("\nMissing dates per symbol in the original dataset:")
print(missing_dates_by_symbol_original)

print("\nMissing dates per symbol in the cleaned dataset:")
print(missing_dates_by_symbol_cleaned)

Total missing dates in the original dataset: 1693
Total missing dates in the cleaned dataset: 1693

Missing dates per symbol in the original dataset:
{'A': 1693, 'AAPL': 1693, 'ABBV': 1693, 'ABNB': 1693, 'ABT': 1693, 'ACGL': 1693, 'ACN': 1693, 'ADBE': 1693, 'ADI': 1693, 'ADM': 1693, 'ADP': 1693, 'ADSK': 1693, 'AEE': 1693, 'AEP': 1693, 'AES': 1693, 'AFL': 1693, 'AIG': 1693, 'AIZ': 1693, 'AJG': 1693, 'AKAM': 1693, 'ALB': 1693, 'ALGN': 1693, 'ALL': 1693, 'ALLE': 1693, 'AMAT': 1693, 'AMCR': 1693, 'AMD': 1693, 'AME': 1693, 'AMGN': 1693, 'AMP': 1693, 'AMT': 1693, 'AMTM': 1693, 'AMZN': 1693, 'ANET': 1693, 'ANSS': 1693, 'AON': 1693, 'AOS': 1693, 'APA': 1693, 'APD': 1693, 'APH': 1693, 'APTV': 1693, 'ARE': 1693, 'ATO': 1693, 'AVB': 1693, 'AVGO': 1693, 'AVY': 1693, 'AWK': 1693, 'AXON': 1693, 'AXP': 1693, 'AZO': 1693, 'BA': 1693, 'BAC': 1693, 'BALL': 1693, 'BAX': 1693, 'BBY': 1693, 'BDX': 1693, 'BEN': 1693, 'BF-B': 1693, 'BG': 1693, 'BIIB': 1693, 'BK': 1693, 'BKNG': 1693, 'BKR': 1693, 'BLDR': 1693

As we can see there are 1693 days missing from our dataset, meaing there no stocks from those days in the dataset independently of the company